# A modular GCD algorithm

The individual steps are the same as in the script. Given are two polynomials $a, b \in \mathbb{Z}[x_1, \dots, x_s][x_n]$ with $0 \leq s < n$.

In [1]:
I.<y,x> = PolynomialRing(QQ, order='lex')

In [2]:
a = 2*x^2*y^3 - x*y^3 + x^3*y^2 + 2*x^4 * y-x^3*y - 6*x*y + 3*y + x^5 - 3*x^2
b = 2*x*y^3 - y^3 - x^2*y^2 + x*y^2 - x^3*y + 4*x*y - 2*y + 2*x^2

$$
a(x, y) = 2 y^{3} x^{2} - y^{3} x + y^{2} x^{3} + 2 y x^{4} - y x^{3} - 6 y x + 3 y + x^{5} - 3 x^{2} \\
b(x, y) = 2 y^{3} x - y^{3} - y^{2} x^{2} + y^{2} x - y x^{3} + 4 y x - 2 y + 2 x^{2}
$$

## Step 0: Univariate polynomials?

$$
\begin{align}
(0) \quad \text{if } s=0 \text{ then } g:=\text{gcd}(\text{cont}(a),\text{cont}(b))\text{GCD_MOD}(\text{pp}(a),\text{pp}(b)); \text{ return } g
\end{align}
$$

In [3]:
a.is_univariate() or b.is_univariate()

False

The polynomials $a$ and $b$ are thus not univariate and Step 0 doesn't apply. But what if it does?

In [4]:
a_uni = 2*x^6 - 13*x^5 + 20*x^4 + 12*x^3 - 20*x^2 - 15*x - 18
b_uni = 2*x^6 + x^5 - 14*x^4 - 11*x^3 + 22*x^2 + 28*x + 8

a_uni.is_univariate() or b.is_univariate()

True

In [5]:
(a_uni / a_uni.content()).gcd(b_uni / b_uni.content()) * a_uni.content().gcd(b_uni.content())

x^2 - x - 2

This is the GCD. So for univariate polynomials we're already finished.

## Step 1: Setting things up

$$
\begin{align}
(1) & \quad & M:=1+\text{min}\left(\text{deg}_{x_s}(a),\text{deg}_{x_s}(b)\right) \\
& \quad & a':=\text{pp}_{x_n}(a) \\
& \quad & b':=\text{pp}_{x_n}(b) \\
& \quad & f:=\text{GCD_MODm}\left(\text{cont}_{x_n}(a),\text{cont}_{x_n}(b),s,s-1\right) \\
& \quad & d:=\text{GCD_MODm}\left(\text{lc}_{x_n}(a'),\text{lc}_{x_n}(b'),s,s-1\right)
\end{align}
$$

In [6]:
xn = I.gens()[0]
xs = I.gens()[1]

xn, xs

(y, x)

In [7]:
M = 1 + min(a.degree(xs), b.degree(xs))
M

4

In [8]:
a_prime = a / gcd(a.polynomial(xn).list())
a_prime

2*y^3*x^2 - y^3*x + y^2*x^3 + 2*y*x^4 - y*x^3 - 6*y*x + 3*y + x^5 - 3*x^2

In [9]:
b_prime = b / gcd(b.polynomial(xn).list())
b_prime

2*y^3*x - y^3 - y^2*x^2 + y^2*x - y*x^3 + 4*y*x - 2*y + 2*x^2

In [10]:
a_prime == a, b_prime == b

(True, True)

In this case, $a'$ and $b'$ are just our original polynomials.

In [11]:
f = gcd(gcd(a.polynomial(xn).list()), gcd(b.polynomial(xn).list()))
f

1

In [12]:
d = gcd(a_prime.polynomial(xn).lc(), b_prime.polynomial(xn).lc())
d

x - 1/2

## Step 2

$$
\begin{align}
(2) & \quad & r:=\text{ an integer s.t. }\text{deg}_{x_n}(a'_{x_{s}-r})=\text{deg}_{x_n}(a')\text{ or }\text{deg}_{x_n}(b'_{x_{s}-r})=\text{deg}_{x_n}(b') \\
& \quad & g'_{(r)}:=\text{GCD_MODm}(a'_{x_{s}-r},b'_{x_{s}-r},n,s-1) \\
& \quad & c:=\text{lc}_{x_n}(g'_{(r)}) \\
& \quad & g_{(r)}:=(d_{x_s-r}\cdot g'_{(r)})/c \quad \text{(but if the division fails, go to (2))}
\end{align}
$$

In [13]:
a_prime.substitute({xs:1})

y^3 + y^2 - 2*y - 2

In [14]:
a_prime.substitute({xs:1}).degree(xn), a_prime.degree(xn)

(3, 3)

In [15]:
b_prime.substitute({xs:1}).degree(xn), b_prime.degree(xn)

(3, 3)

So we can choose $r=1$. To do this more systematically (also to reuse code later), let's write a class that remembers $r$ and in which we can build in a couple safeguarding features.

In [16]:
class r_int():
    def __init__(self, r_max=10000):
        self.r = 0
        self.r_max = r_max
    
    def reset_r(self):
        self.r = 0
        
    def get_new_r(self, a_prime, b_prime, xs, xn):
        while self.r < self.r_max:
            self.r += 1
            if a_prime.substitute({xs:self.r}).degree(xn) == a_prime.degree(xn):
                return self.r
            if b_prime.substitute({xs:self.r}).degree(xn) == b_prime.degree(xn):
                return self.r
        raise Exception("r exceeded r_max")

In [17]:
variable_r = r_int()
variable_r.get_new_r(a_prime, b_prime, xs, xn)

1

In [18]:
variable_r.r

1

In [19]:
g_prime = a_prime.substitute({xs:variable_r.r}).gcd(b_prime.substitute({xs:variable_r.r}))
g_prime

y + 1

In [20]:
c = g_prime.polynomial(xn).lc()
c

1

In [21]:
g_r = (d.substitute({xs:variable_r.r}) * g_prime) / c
g_r

1/2*y + 1/2

If the division fails, we're supposed to go to (2). We can test that with ```quo_rem()```. The division definitely fails when the quotient is 0.

In [22]:
(d.substitute({xs:variable_r.r}) * g_prime).quo_rem(c)

(1/2*y + 1/2, 0)

## Step 3

$$
\begin{align}
(2) & \quad & m:=1 \\
& \quad & g:=g_{(r)} \\
\end{align}
$$

In [23]:
m = 1
m

1

In [24]:
g = g_r
g

1/2*y + 1/2

## Step 4

$$
\begin{align}
(2) &  & \text{while }m\leq M\text{ do} & \\
&  &  & r:=\text{ a new integer s.t. }\text{deg}_{x_n}(a'_{x_{s}-r})=\text{deg}_{x_n}(a')\text{ or }\text{deg}_{x_n}(b'_{x_{s}-r})=\text{deg}_{x_n}(b') \\
& & & g'_{(r)}:=\text{GCD_MODm}(a'_{x_{s}-r},b'_{x_{s}-r},n,s-1) \\
& & & c:=\text{lc}_{x_n}(g'_{(r)}) \\
& & & g_{(r)}:=(d_{x_s-r}\cdot g'_{(r)})/c \quad \text{(but if the division fails, continue)} \\
& & & \text{if }\text{deg}_{x_n}(g_{(r)})<\text{deg}_{x_n}(g)\text{ then goto (3))} \\
& & & \text{if }\text{deg}_{x_n}(g_{(r)})=\text{deg}(g) \\
& & & \text{then incorporate }g_{(r)}\text{ into }g\text{ by Newton interpolation} \\
& & & m:=m+1
\end{align}
$$

We'll do the while-loop later. Let's just do the steps manually for now.

In [25]:
variable_r.get_new_r(a_prime, b_prime, xs, xn)

2

In [26]:
g_prime = a_prime.substitute({xs:variable_r.r}).gcd(b_prime.substitute({xs:variable_r.r}))
g_prime

3*y + 4

In [27]:
c = g_prime.polynomial(xn).lc()
c

3

In [28]:
g_r = (d.substitute({xs:variable_r.r}) * g_prime) / c
g_r

3/2*y + 2

In [29]:
g_r.degree(xn) < g.degree(xn)

False

So we don't have to go to (3) in this case.

In [30]:
g_r.degree(xn) == g.degree(xn)

True

In [31]:
g_r, g

(3/2*y + 2, 1/2*y + 1/2)

In [32]:
def interpolate(r1, poly1, r2, poly2, I, var):
    from sympy.polys.polyfuncs import interpolate
    import sympy
    
    poly1s = sympy.sympify(poly1.__repr__())
    poly2s = sympy.sympify(poly2.__repr__())
    
    symb = sympy.Symbol(var.__repr__())
    
    result = interpolate([(r1, poly1s), (r2, poly2s)], symb)
    
    return I(result)

In [33]:
interim_result = interpolate(1, g, 2, g_r, I, xs)
interim_result

y*x - 1/2*y + 3/2*x - 1

In [34]:
g.quo_rem(interim_result)

(0, 1/2*y + 1/2)

So this is not the gcd and the algorithm continues, i.e. we set $m:=m+1$ and start again at (4). Note also that we have to save each $g$ for the interpolation later, so we'll have to modify our function ```interpolate``` to accept more than two polynomials in the future.

In [46]:
m = m + 1
m, M

(3, 4)

In [35]:
gs = [(1, g), (2, g_r)]
gs

[(1, 1/2*y + 1/2), (2, 3/2*y + 2)]

Saving the $g$'s in there so we can pass them directly to a future interpolation function. So we start (4) anew.

In [36]:
variable_r.get_new_r(a_prime, b_prime, xs, xn)

3

In [37]:
g_prime = a_prime.substitute({xs:variable_r.r}).gcd(b_prime.substitute({xs:variable_r.r}))
g_prime

5*y + 9

In [38]:
c = g_prime.polynomial(xn).lc()
c

5

In [39]:
g_r = (d.substitute({xs:variable_r.r}) * g_prime) / c
g_r

5/2*y + 9/2

In [40]:
g_r.degree(xn) < g.degree(xn)

False

So again we don't have to go to step (3).

In [41]:
g_r.degree(xn) == g.degree(xn)

True

In [42]:
gs.append((variable_r.r, g_r))
gs

[(1, 1/2*y + 1/2), (2, 3/2*y + 2), (3, 5/2*y + 9/2)]

In [43]:
def interpolate(rpolys, I, var):
    from sympy.polys.polyfuncs import interpolate
    import sympy
    
    symb = sympy.Symbol(var.__repr__())
    
    rpolys_sympified = [(gr[0], sympy.sympify(gr[1].__repr__())) for gr in rpolys]
    
    result = interpolate(rpolys_sympified, symb)
    
    return I(result)

In [44]:
interim_result = interpolate(gs, I, xs)
interim_result

y*x - 1/2*y + 1/2*x^2

In [49]:
a.quo_rem(interim_result), b.quo_rem(interim_result)

((2*y^2*x + 2*x^3 - 6, 0), (2*y^2 - 2*y*x + 4, 0))

So this is the GCD. Actually, the algorithm would continue one more step until $m=M$.

## Step 5: Wrapping up or starting anew

$$
\begin{align}
(5) &  & & g:=f\cdot\text{pp}_{x_n}(g) \\
&  &  & \text{if }g\in\mathbb{Z}[x_1,\dots,x_s][x_n]\text{ and }g|a\text{ and }g|b\text{ then return }g \\
& & & \text{goto }(2) \\
\end{align}
$$

In [52]:
def pp(poly, var):
    return poly / gcd(poly.polynomial(var).list())

In [53]:
pp(interim_result, xn)

y*x - 1/2*y + 1/2*x^2

In [55]:
g_final = f * pp(interim_result, xn)
g_final

y*x - 1/2*y + 1/2*x^2

In [56]:
a.quo_rem(g_final)[1] == 0, b.quo_rem(g_final)[1] == 0

(True, True)

This is the final result and terminates the algorithm. If it weren't, we'd have to go back to (2).